In [154]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [281]:
import pandas as pd
import numpy as np
np.random.seed(42)
import re
from pprint import pprint
import ast

import matplotlib.pyplot as plt
%matplotlib inline

---

### clean text

In [3]:
from nltk.corpus import stopwords
#pip install razdel
from razdel import tokenize # https://github.com/natasha/razdel
#pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [4]:
stopword_ru = stopwords.words('russian')
with open('preprocessing/stopwords.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stopword_ru.append(w)
print(len(stopword_ru))

205


In [5]:
def clean_text(text):
    '''
    очистка текста
        
    на выходе очищеный текст
    '''
    
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')

    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    return text

---

### read data

In [6]:
%%time

df=pd.read_csv('data/news_lenta.csv')
print(df.shape)

(699777, 5)
CPU times: user 23.3 s, sys: 7.72 s, total: 31 s
Wall time: 33.1 s


In [7]:
df.head(2)

,tags,text,title,topic,url
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/


### leave only [`url`,`text`] columns

In [8]:
df=df[['url','text']].copy()
df.head(2)

,url,text
0,https://lenta.ru/news/2018/07/22/usik_plany/,"Украинский боксер Александр Усик рассказал, чт..."
1,https://lenta.ru/news/2018/07/22/evacuation/,Израиль эвакуировал из Сирии активистов «Белых...


clean text

In [9]:
%%time
df['text_clean']=df.text.apply(lambda x: clean_text(x))

CPU times: user 6min 37s, sys: 20.5 s, total: 6min 58s
Wall time: 7min 27s


In [10]:
df.head()

,url,text,text_clean
0,https://lenta.ru/news/2018/07/22/usik_plany/,"Украинский боксер Александр Усик рассказал, чт...",украинский боксер александр усик рассказал что...
1,https://lenta.ru/news/2018/07/22/evacuation/,Израиль эвакуировал из Сирии активистов «Белых...,израиль эвакуировал из сирии активистов белых ...
2,https://lenta.ru/news/2018/07/22/hazard_real/,Лондонский «Челси» отказался продать мадридско...,лондонский челси отказался продать мадридскому...
3,https://lenta.ru/news/2018/07/22/otvet/,Вице-президент Федерации профессионального бок...,вицепрезидент федерации профессионального бокс...
4,https://lenta.ru/news/2018/07/22/moneyrich/,Журнал Billboard опубликовал рейтинг самых выс...,журнал billboard опубликовал рейтинг самых выс...


In [11]:
%%time
df[['url','text_clean']].to_csv('data/news_lenta_clean.csv',index=False)

CPU times: user 36.2 s, sys: 43.5 s, total: 1min 19s
Wall time: 1min 43s


In [6]:
df = pd.read_csv('data/news_lenta_clean.csv')
print(df.shape)
df.head(2)

(699777, 2)


,url,text_clean
0,https://lenta.ru/news/2018/07/22/usik_plany/,украинский боксер александр усик рассказал что...
1,https://lenta.ru/news/2018/07/22/evacuation/,израиль эвакуировал из сирии активистов белых ...


---

### take our dataset with some names like `cleaned names`

In [453]:
cleaned_name = pd.read_csv('data/cleaned_names.csv')
print(cleaned_name.shape)
cleaned_name.head()

(2201, 5)


,cleaned_name,has_namesake,names( from text),cases,surname_cases
0,аба бакр аль-багдади,False,"Абу Бакр аль-Багдади,Абу Бакра Аль-Багдади,Абу...","аба бакр аль-багдади,абе бакр альбагдад,абой б...","аль-багдади, альбагдад"
1,абд раббо мансур хади,False,"Абд Раббо Мансур Хади,Абд Раббо Мансура Хади","абде раббо мансур хади, абдом раббо мансур ха...",хади
2,абдулла абдулла,False,"Абдулла Абдулла,Абдулле Абдулле,Абдуллой Абдул...","абдулла абдулла,абдулле абдулле,абдуллой абдул...","абдулла, абдулле, абдуллой, абдуллу"
3,абдулла гюль,False,"Абдулла Гюль,Абдулле Гюлю,Абдуллу Гюля,Абдуллы...","абдулла гюле,абдулла гюлем,абдулла гюль,абдулл...","гюле, гюлем, гюль, гюлю"
4,абубакар шекау,False,"Абубакар Шекау,Абубакара Шекау","абубакар шекау,абубакара шекау,абубакаре шекау...",шекау


concat 2 columns `cases` & `surname cases`

In [454]:
#cleaned_name['cases'] = cleaned_name['cases']+','+cleaned_name['surname_cases']
cleaned_name.head()

,cleaned_name,has_namesake,names( from text),cases,surname_cases
0,аба бакр аль-багдади,False,"Абу Бакр аль-Багдади,Абу Бакра Аль-Багдади,Абу...","аба бакр аль-багдади,абе бакр альбагдад,абой б...","аль-багдади, альбагдад"
1,абд раббо мансур хади,False,"Абд Раббо Мансур Хади,Абд Раббо Мансура Хади","абде раббо мансур хади, абдом раббо мансур ха...",хади
2,абдулла абдулла,False,"Абдулла Абдулла,Абдулле Абдулле,Абдуллой Абдул...","абдулла абдулла,абдулле абдулле,абдуллой абдул...","абдулла, абдулле, абдуллой, абдуллу"
3,абдулла гюль,False,"Абдулла Гюль,Абдулле Гюлю,Абдуллу Гюля,Абдуллы...","абдулла гюле,абдулла гюлем,абдулла гюль,абдулл...","гюле, гюлем, гюль, гюлю"
4,абубакар шекау,False,"Абубакар Шекау,Абубакара Шекау","абубакар шекау,абубакара шекау,абубакаре шекау...",шекау


leave raws where mark `has_namesake = False`

In [455]:
cleaned_name = cleaned_name[['has_namesake','cleaned_name','cases']].copy()
cleaned_name = cleaned_name[cleaned_name.has_namesake == 0]
print(cleaned_name.shape)
cleaned_name.head()

(1788, 3)


,has_namesake,cleaned_name,cases
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б..."
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха..."
2,False,абдулла абдулла,"абдулла абдулла,абдулле абдулле,абдуллой абдул..."
3,False,абдулла гюль,"абдулла гюле,абдулла гюлем,абдулла гюль,абдулл..."
4,False,абубакар шекау,"абубакар шекау,абубакара шекау,абубакаре шекау..."


### edit cleaned_name
* leave only the 1st value if we have they a few
* join via `_` splitting `,`

In [456]:
cleaned_name['cleaned_name_new'] = cleaned_name.cleaned_name\
                                   .apply(lambda x: '_'.join(x.split(',')[0]\
                                                              .split(' ')))
cleaned_name.head()

,has_namesake,cleaned_name,cases,cleaned_name_new
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б...",аба_бакр_аль-багдади
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха...",абд_раббо_мансур_хади
2,False,абдулла абдулла,"абдулла абдулла,абдулле абдулле,абдуллой абдул...",абдулла_абдулла
3,False,абдулла гюль,"абдулла гюле,абдулла гюлем,абдулла гюль,абдулл...",абдулла_гюль
4,False,абубакар шекау,"абубакар шекау,абубакара шекау,абубакаре шекау...",абубакар_шекау


### edit cases

create the list all possible mentions dropping whitespace name

drop null values

In [458]:
cleaned_name = cleaned_name[cleaned_name.cases.notnull()]

create list

In [459]:
cleaned_name['cases_new'] = cleaned_name.cases.apply(lambda x: [i.strip() for i in x.split(',')])
cleaned_name.head()

,has_namesake,cleaned_name,cases,cleaned_name_new,cases_new
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б...",аба_бакр_аль-багдади,"[аба бакр аль-багдади, абе бакр альбагдад, або..."
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха...",абд_раббо_мансур_хади,"[абде раббо мансур хади, абдом раббо мансур ха..."
2,False,абдулла абдулла,"абдулла абдулла,абдулле абдулле,абдуллой абдул...",абдулла_абдулла,"[абдулла абдулла, абдулле абдулле, абдуллой аб..."
3,False,абдулла гюль,"абдулла гюле,абдулла гюлем,абдулла гюль,абдулл...",абдулла_гюль,"[абдулла гюле, абдулла гюлем, абдулла гюль, аб..."
4,False,абубакар шекау,"абубакар шекау,абубакара шекау,абубакаре шекау...",абубакар_шекау,"[абубакар шекау, абубакара шекау, абубакаре ше..."


---

## test some approaches to find some words in the article & replace them

* ### re

In [110]:
text='abc dd acc'
re.sub('(abc|dd)', '**', text)

'** ** acc'

* ### ahocorasick

In [112]:
# https://github.com/WojciechMula/pyahocorasick/#examples
# pip install pyahocorasick
import ahocorasick 

In [113]:
A = ahocorasick.Automaton()

In [115]:
for index, word in enumerate('he her hers she'.split()):
    A.add_word(word, (index, word))

A.make_automaton()

In [118]:
A.get('he')

(0, 'he')

In [119]:
A.

(2, 'hers')

In [130]:
[i for i in A.keys()]

['she', 'he', 'her', 'hers']

In [117]:
[item for item in A.iter('_hershe_')]

[(2, (0, 'he')),
 (3, (1, 'her')),
 (4, (2, 'hers')),
 (6, (3, 'she')),
 (6, (0, 'he'))]

=> the don't have replace

* ### flashtext

In [251]:
# https://github.com/vi3k6i5/flashtext
# !pip install flashtext
from flashtext import KeywordProcessor

* 0

In [144]:
del keyword_processor
keyword_processor = KeywordProcessor()

In [150]:
keyword_processor.add_keyword('Apple Inc.','Apple')

True

In [151]:
text1 = '''Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services.'''

In [152]:
keywords_found = keyword_processor.extract_keywords(text1)
keywords_found

['Apple']

In [153]:
new_sentence = keyword_processor.replace_keywords(text1)
new_sentence

'Apple is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services.'

* 1

In [172]:
del keyword_processor
keyword_processor = KeywordProcessor()

In [173]:
keyword_dict = {"*apple*": ["Apple Inc.", "Apple"],
                "*california*": ["Cupertino"]
               }
keyword_processor.add_keywords_from_dict(keyword_dict)

In [174]:
text2 = '''Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services.'''

In [175]:
keywords_found = keyword_processor.extract_keywords(text2)
keywords_found

['*apple*', '*california*']

In [176]:
new_sentence = keyword_processor.replace_keywords(text2)
new_sentence

'*apple* is an American multinational technology company headquartered in *california*, California, that designs, develops, and sells consumer electronics, computer software, and online services.'

---

---

### create dict from our dataset where key is `cleaned_name_new` & values - `cases_new`

In [477]:
print(cleaned_name.shape)
cleaned_name.head(2)

(1788, 5)


,has_namesake,cleaned_name,cases,cleaned_name_new,cases_new
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б...",аба_бакр_аль-багдади,"[аба бакр аль-багдади, абе бакр альбагдад, або..."
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха...",абд_раббо_мансур_хади,"[абде раббо мансур хади, абдом раббо мансур ха..."


In [478]:
%%time

dict_cleaned_name={}
for _,i in cleaned_name.iterrows():
    dict_cleaned_name[f'{i.cleaned_name_new}']=i.cases_new
    
print(len(dict_cleaned_name))

1788
CPU times: user 254 ms, sys: 3.86 ms, total: 258 ms
Wall time: 258 ms


In [487]:
dict_cleaned_name['виктор_янукович']

['виктор янукович',
 'виктора януковича',
 'викторе януковиче',
 'виктором януковичем',
 'виктору януковичу']

### try flashtext

In [495]:
del kp
kp = KeywordProcessor()

kp.add_keywords_from_dict(dict_cleaned_name)
len(kp)

8474

In [496]:
text = '''абромавичус увидел гитлера и авраама линкольна врезав алешине'''
print(kp.extract_keywords(text))
print(kp.replace_keywords(text))

['авраам_линкольн']
абромавичус увидел гитлера и авраам_линкольн врезав алешине


### replace words in our dataset

In [497]:
df.head()

,url,text_clean
0,https://lenta.ru/news/2018/07/22/usik_plany/,украинский боксер александр усик рассказал что...
1,https://lenta.ru/news/2018/07/22/evacuation/,израиль эвакуировал из сирии активистов белых ...
2,https://lenta.ru/news/2018/07/22/hazard_real/,лондонский челси отказался продать мадридскому...
3,https://lenta.ru/news/2018/07/22/otvet/,вицепрезидент федерации профессионального бокс...
4,https://lenta.ru/news/2018/07/22/moneyrich/,журнал billboard опубликовал рейтинг самых выс...


In [499]:
%%time
df['text_clean_name'] = df.text_clean.apply(lambda x: kp.replace_keywords(x))

CPU times: user 28min 37s, sys: 31 s, total: 29min 8s
Wall time: 29min 46s


In [218]:
df.text_clean[:10].apply(lambda x: keyword_processor.replace_keywords(x))

0    украмун_чжэ_инский боксер алексвиктор_андр уси...
1    израиль эвакуким_чен_ировал из ским_чен_ирии а...
2    лондонский челси отказался продать мадридскому...
3    вицепрезидент федерации профессиоли_нального б...
4    журли_нал billboard опубликовал рейтмун_чжэ_ин...
5    бывшего участника сборной комвиктор_анды актоб...
6    великобритвиктор_ания откажется от выплат отст...
7    талисмвиктор_аны олимпийских и паралимпийских ...
8    около российской базы хмеймим в ским_чен_ирии ...
9    президент украмун_чжэ_ины петр_порошенко поздр...
Name: text_clean, dtype: object

In [500]:
df.text_clean.head(10)

0    украинский боксер александр усик рассказал что...
1    израиль эвакуировал из сирии активистов белых ...
2    лондонский челси отказался продать мадридскому...
3    вицепрезидент федерации профессионального бокс...
4    журнал billboard опубликовал рейтинг самых выс...
5    бывшего участника сборной команды актобе квн д...
6    великобритания откажется от выплат отступных е...
7    талисманы олимпийских и паралимпийских игр в т...
8    около российской базы хмеймим в сирии произошл...
9    президент украины петр порошенко поздравил бок...
Name: text_clean, dtype: object